# SRL Patient recognition experiments

In this notebook I carry out experiments to test whether two Semantic Role Labelling (SRL) systems can correctly identify patients in sentences with varying structures. This code was based on code provided by Pia Sommerauer.

In this code I load two models, namely the AllenNLP SRL model and the AllenNLP SRL BERT model. I create a variety of tets cases, for wich I evaluate the performance of the two models. All the test sentences are stored in a json file specified through the `test_sents_path` variable. The SRL predictions are stored in the json file specified through `srl_pred_path`, and similarly the SRL BERT predictions are stored at the path `bert_pred_path`.

### Recognizing the difference between an instrument and a description
* Instrument: 'John hit the door with a hammer'
* Description : 'John hit the door with the poster'


In [4]:
from allennlp_models.pretrained import load_predictor

In [5]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [6]:
from checklist.pred_wrapper import PredictorWrapper

In [3]:
import json
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [1]:
from utils_functions import *

### Load the models 

In [ ]:
# load the regular SRL model
srl_predictor = load_predictor('structured-prediction-srl')
# load the SRL BERT model
srlbert_predictor = load_predictor('structured-prediction-srl-bert')

In [8]:
#functions to create model predictions for a list containing sentences
### added by pia, edited by Goya ###

def predict_srl(data):
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred


def predict_srlbert(data):
    pred = []
    for d in data:
        pred.append(srlbert_predictor.predict(d))
    return pred

predict_srl = PredictorWrapper.wrap_predict(predict_srl)
predict_srlbert = PredictorWrapper.wrap_predict(predict_srlbert)

### Define output file paths

In [12]:
#create lists to store test sentences and model predictions in 
test_data = []
SRLBERT_predictions = []
SRL_predictions = []

In [9]:
#define paths to output files
test_sents_path = './JSON_test_and_predict_files/test_data_instrument.json'
bert_pred_path = './JSON_test_and_predict_files/BERT_predictions_instrument.json'
srl_pred_path = './JSON_test_and_predict_files/SRL_predictions_instrument.json'

#set name of current capability
capability = 'instrument_recognition'

### Load Checklist tests (Load functions defined in utils)
Load functions to test whether the arguments are classified correctly

In [2]:
expect_arg1_verb0 = Expect.single(found_door_arg1_verb0)
expect_arg2_verb0 = Expect.single(found_door_arg2_verb0)
expect_arg2_verb1 = Expect.single(found_door_arg2_verb1)

### Load wordlists to use in sample sentences

In [10]:
editor = Editor()
instrument = ['hammer', 'knife', 'stone', 'bottle', 'table', 'chair', 'racket']
description = ['poster', 'flag', 'window', 'bell', 'paint', 'holes', 'mailbox', 'nameplate']

## Agent recognition : small and large predicate distance

In [13]:
#create samples
testcase_name = 'instrument'
t = editor.template("{first_name} hit the door with the {instrument}", instrument=instrument, meta=True, nsamples=100)

#make and store predictions for the two models
test_data, SRL_predictions, SRLBERT_predictions = predict_and_store(t, capability, testcase_name, \
                                                                    expect_arg2_verb0, format_srl_verb0, \
                                                                    predict_srl, predict_srlbert, test_data, \
                                                                    SRL_predictions, SRLBERT_predictions)

SRL
Predicting 100 examples
Test cases:      100
Fails (rate):    1 (1.0%)

Example fails:
[ARG0: Dick] [V: hit] [ARG1: the door] [ARGM-MNR: with the knife]
----
SRL BERT
Predicting 100 examples
Test cases:      100
Fails (rate):    17 (17.0%)

Example fails:
[ARG0: Harry] [V: hit] [ARG1: the door] [ARGM-MNR: with the racket]
----
[ARG0: Linda] [V: hit] [ARG1: the door] [ARGM-MNR: with the racket]
----
[ARG0: Steven] [V: hit] [ARG1: the door] [C-ARG1: with the table]
----


In [14]:
#create samples
testcase_name = 'patient'
t = editor.template("{first_name} hit the door with the {description}", description=description, meta=True, nsamples=100)

#make and store predictions for the two models
test_data, SRL_predictions, SRLBERT_predictions = predict_and_store(t, capability, testcase_name, \
                                                                    expect_arg1_verb0, format_srl_verb0, \
                                                                    predict_srl, predict_srlbert, test_data, \
                                                                    SRL_predictions, SRLBERT_predictions)


SRL
Predicting 100 examples
Test cases:      100
Fails (rate):    100 (100.0%)

Example fails:
[ARG0: Howard] [V: hit] [ARG1: the door] [ARG2: with the window]
----
[ARG0: Alexandra] [V: hit] [ARG1: the door] [ARG2: with the mailbox]
----
[ARG0: Howard] [V: hit] [ARG1: the door] [ARG2: with the holes]
----
SRL BERT
Predicting 100 examples
Test cases:      100
Fails (rate):    100 (100.0%)

Example fails:
[ARG0: Howard] [V: hit] [ARG1: the door] [ARG2: with the window]
----
[ARG0: Bill] [V: hit] [ARG1: the door] [ARG2: with the bell]
----
[ARG0: Annie] [V: hit] [ARG1: the door] [ARG2: with the window]
----


### Store all data to JSON

In [15]:
#store the test sentences
store_data(test_sents_path, test_data, new_file=True)
#store the model predictions
store_data(bert_pred_path, SRLBERT_predictions, new_file=True)
store_data(srl_pred_path, SRL_predictions, new_file=True)